# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на новые тарифы. 

Задача: построить модель для задачи классификации, которая выберет подходящий тариф. 
Построить модель с максимально большим значением *accuracy*. Минимум 0.75. Проверить *accuracy* на тестовой выборке.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.dummy import DummyClassifier
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [2]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [18]:
df.duplicated().sum()

0

## Разбейте данные на выборки

In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']


Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разобьем в соотношении 3:1:1 (обучающая-60%, тестовая и валидационная по 20%)

In [6]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.4, 
                                                                            random_state=12345
                                                                           ) 

In [7]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, 
                                                                            target_test, 
                                                                            test_size=0.5, 
                                                                            random_state=12345
                                                                           ) 

In [8]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


In [9]:
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


In [10]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


## Исследуйте модели

**Дерево решений**

In [19]:
best_result = 0
best_depth=0
for depth in range(1, 6):
    model =DecisionTreeClassifier(random_state=12345,max_depth=depth) 
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    result =accuracy_score(predictions_valid, target_valid)
    print(result)
    if result > best_result:
        best_result = result
        best_depth = depth
print("Лучший результат дерева решений:", best_result)
print("Лучшая глубина дерева решений:", best_depth)

0.7542768273716952
0.7822706065318819
0.7853810264385692
0.7791601866251944
0.7791601866251944
Лучший результат дерева решений: 0.7853810264385692
Лучшая глубина дерева решений: 3


При исследовании 5 вариантов глубины дерева лучший результат по модели равен 0.79, наилучший результат достигается при глубине = 3

**Логистическая регрессия**

In [12]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
accuracy = accuracy_score(predictions_valid, target_valid)
print('Качество логистической регрессии:',  accuracy)

Качество логистической регрессии: 0.7107309486780715


Лучший результат равен 0.71

**Случайный лес**

In [25]:
best_result = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range(1, 14, 2):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train,target_train)
        predictions_valid = model.predict(features_valid)
        accuracy =accuracy_score(predictions_valid, target_valid)
        if accuracy > best_result:
            best_result = accuracy
            best_depth = depth
            best_est = est
        
print("Лучший результат случайного леса:", best_result)
print("Лучшая глубина случайного леса:", best_depth)
print ("Количество деревьев (эстиматоров)", est)

Лучший результат случайного леса: 0.80248833592535
Лучшая глубина случайного леса: 7
Количество деревьев (эстиматоров) 100


При исследовании 5 вариантов глубины дерева лучший результат по модели равен 0.80 при глубине = 7.

Данная модель показала себя наиболее эффективной среди протестированных и рекомендуется к внедрению. Рекомендуемые гиперпараметры:
- n_estimators = 100
- max_depth = 7.


При данных гиперпараметрах получим accuracy = 0.80



## Проверьте модель на тестовой выборке

**Проверим внедряемую модель "Случайный лес" на тестовой выборке**

In [27]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions_train = model.predict(features_train)

accuracy = accuracy_score(target_train, predictions_train)  

predictions_test = model.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test)

print("Качество случайного дерева для тестовой выборки:", accuracy_test)

Качество случайного дерева для тестовой выборки: 0.7853810264385692


Результат проверки на тестовой выборке равен 0.79, что близко к результатам проверки на валидационной выборке.

## (бонус) Проверьте модели на адекватность

In [31]:
dummy_clf = DummyClassifier(strategy='constant', constant=0)
dummy_clf.fit(features_test, target_test)
accuracy_score(dummy_clf.predict(features_test), target_test)

0.6842923794712286

После обучения модели можно провести тестирование и результаты использовать для сравнения с результатом выбранной модели. Результаты DummyClassifier значительно ниже результатов всех моделей,использованных в работе, значит, модели тест на адекватность прошли

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

## Итоговый вывод

   В ходе исследования были загружены и изучены данные о поведении клиентов. Эти данные были разбиты на 3 выборки: обучающую, валидационную и тестовую в соотношении 3:1:1 (обучающая-60%, тестовая и валидационная по 20%).

  Были проверены три модели: дерево решений, логистическая регрессия и случайный лес. Использовались 5 значений глубины для двух моделей. 
  
  При исследовании 5 вариантов глубины дерева лучший результат по модели равен 0.80 при глубине = 7.

Данная модель показала себя наиболее эффективной среди протестированных и рекомендуется к внедрению при следующих гиперпараметрах:
- n_estimators = 100
- max_depth = 7.


При данных гиперпараметрах получим accuracy = 0.80


  Так же была выполнена и пройдены проверка на адекватность.